In [1]:
import pandas as pd

In [39]:
df_raw = pd.read_csv("data/ppp_loan_strip.csv", index_col=0)

In [40]:
len(df_raw)

661218

In [12]:
df.isnull().sum()

LoanRange           0
BusinessName        8
Address            17
City               15
State               0
Zip                16
NAICSCode        6783
BusinessType     1429
JobsRetained    40506
dtype: int64

### Save and drop nulls
Also saving csv's for loans with zero jobs retained reported

In [22]:
na_addr = df[df['Address'].isna()]
na_city = df[df['City'].isna()]
na_zip = df[df['Zip'].isna()]
na_jobs = df[df['JobsRetained'].isna()]
zero_jobs = df[df['JobsRetained'] == 0]
xx_state = df[df['State'] == "XX"]

In [38]:
na_city.shape, na_addr.shape, na_zip.shape, na_jobs.shape, zero_jobs.shape

((15, 9), (17, 9), (16, 9), (40506, 9), (48922, 9))

In [41]:
no_jobs = (len(na_jobs) + len(zero_jobs))/len(df_raw)
no_jobs

0.13524737681067364

There are 40,506 loans associated with no jobs retained reported and 48,922 loans associated with zero jobs retained reported. This accounts for over 13% of loans.

In [14]:
dropped = [na_addr, na_city, na_zip, na_jobs]

In [25]:
na_addr.to_csv("data/na_addr.csv")
na_city.to_csv("data/na_city.csv")
na_zip.to_csv("data/na_zip.csv")
na_jobs.to_csv("data/na_jobs.csv")
zero_jobs.to_csv("data/zero_jobs.csv")
xx_state.to_csv("data/xx_state.csv")

In [26]:
df = df.dropna()

## Convert loan range column to single-letter code
There are five types:
- A: \$5-10 million
- B: \$2-5 million
- C: \$1-2 million
- D: \$350,000-1 million
- E: \$150,000-350,000

Drop loan range column.

In [30]:
import re

In [34]:
df['LoanCat'] = df.apply(lambda row: re.search(r"[abcde]", row['LoanRange'])[0], axis=1)
df = df.drop(columns="LoanRange")

In [58]:
pickle.dump(df, open("data/loans_clean.pkd", "wb"))

### Save csv's for each loan category

In [49]:
loan_type_list = ['a', 'b', 'c', 'd', 'e']

for l_type in loan_type_list:
    df[df['LoanCat'] == l_type].to_csv('data/loan_cat_' + l_type.capitalize() + '.csv')

## Convert address to full address for geocoding
`.split(',')` removes address appendices such as "Suite A", "Room 17", etc.

In [43]:
df['full_addr'] = df.apply(lambda row: (row['Address'].split(',')[0] + ', ' 
                            + row['City'] + ', ' + row['State']), axis=1)

### Save csv's for individual loan categories with full addresses

In [50]:
loan_type_list = ['a', 'b', 'c', 'd', 'e']

for l_type in loan_type_list:
    df[df['LoanCat'] == l_type].to_csv('data/addr_loan_cat_' + l_type.capitalize() + '.csv')

In [51]:
import pickle

In [52]:
for l_type in loan_type_list:
    pickle.dump(df[df['LoanCat'] == l_type], open("data/addr_loan_cat_" + l_type.capitalize() + ".pkd", "wb"))